In [23]:
%pip install langchain pinecone-client faiss-cpu -q
%pip install langchain_community -q
%pip install ipywidgets -q
%pip install sentence-transformers -q
%pip install langchain_pinecone -q
%pip install langchain_openai -q


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is a

In [1]:
import pandas as pd
import numpy as np
import time
import json
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone, FAISS
from langchain.text_splitter import CharacterTextSplitter
import pinecone
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.schema import Document

In [2]:
def load_api_key(key):
    path = "/Users/khushalidaga/vs-code/capstone/openai_api.json"
    with open(path, "r") as file:
        config = json.load(file)
    return config[key]

In [3]:
def get_word2vec():
    path = "/Users/khushalidaga/vs-code/capstone/metadata/bot_word2vec_representations.json"
    with open(path, 'r') as f:
        agent_words = json.load(f)
    return agent_words

In [4]:
# get agent words
agent_words = get_word2vec()

#  get pincone api key
pinecone_key = load_api_key("PINECONE_API_KEY")

os.environ['PINECONE_API_KEY'] = pinecone_key

# Initialize the embeddings model
model_name = 'multilingual-e5-large'
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=pinecone_key
)

In [8]:
# Initialize Pinecone or FAISS vector database
# For Pinecone:
pc = Pinecone(api_key=pinecone_key)
# Define the index settings
index_name = "vector-index"
dimension = 1024  # Dimension for 'all-MiniLM-L6-v2' embeddings model output

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embeddings.dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
            ) 
    )
    # Wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# Access the index
vector_db = pc.Index(index_name)
# See that it is empty
print("Index before upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")

Index before upsert:
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}




In [38]:
# Flatten the terms and scores into a format suitable for the Pinecone upsert
documents = []
for doc_name, word_weights in agent_words.items():
    # Create a single string for each document by combining the word and its weight
    doc = Document(
        page_content=", ".join(word_weights.keys()),  # Text could be a concatenation of words
        metadata={"bot_name": doc_name}  # Add any other relevant metadata if needed
    )
    documents.append(doc)

print(documents)

[Document(metadata={'bot_name': 'Customer Database Search'}, page_content='total, decisions, channels, average, metric, measuring, reveal, dividing, increase, brand, offers, purchases, calculate, behavioral, product'), Document(metadata={'bot_name': 'Organizational Information'}, page_content='hires, requesting, schedule, positive, providing, job, overview'), Document(metadata={'bot_name': 'Internet Search'}, page_content='growing, patients, analyze, consumers, adopt, commerce, working, reducing, latest, power, rise, environmentally, intelligence, presents, artificial')]


In [39]:
namespace = "wondervector5000"

docsearch = PineconeVectorStore.from_documents(
    documents=documents,
    index_name=index_name,
    embedding=embeddings,
    namespace=namespace
)

time.sleep(5)

# See how many vectors have been upserted
print("Index after upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")
time.sleep(2)


Index after upsert:
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'wondervector5000': {'vector_count': 9}},
 'total_vector_count': 9}




In [46]:
# Perform similarity search with scores
query = "Can you provide a list of customers who've been featured in our marketing materials?"
results = docsearch.similarity_search_with_score(query, k=5)

# Print results
print(f"Query: {query}\n")
print("Results:")
for doc, score in results:
    print(f"Bot: {doc.metadata['bot_name']}")
    print(f"Similarity Score: {score}")
    print(f"Content: {doc.page_content[:50]}...")  # Truncate content for readability
    print()

Query: Can you provide a list of customers who've been featured in our marketing materials?

Results:
Bot: Customer Database Search
Similarity Score: 0.813858
Content: total, decisions, channels, average, metric, measu...

Bot: Customer Database Search
Similarity Score: 0.812541604
Content: total decisions channels average metric measuring ...

Bot: Organizational Information
Similarity Score: 0.808923662
Content: hires, requesting, schedule, positive, providing, ...

Bot: Internet Search
Similarity Score: 0.808375895
Content: growing, patients, analyze, consumers, adopt, comm...

Bot: Internet Search
Similarity Score: 0.80659622
Content: growing patients analyze consumers adopt commerce ...



In [52]:
# Calculate the average similarity score for each bot
score_dict = {}
for doc, score in results:
    bot = doc.metadata['bot_name']
    if bot not in score_dict:
        score_dict[bot] = {'total_score': 0, 'count': 0}
    score_dict[bot]['total_score'] += score
    score_dict[bot]['count'] += 1

# Calculate averages
average_scores = {bot: data['total_score'] / data['count'] for bot, data in score_dict.items()}

# Find the bot with the highest average score
best_bot = max(average_scores, key=average_scores.get)
best_score = average_scores[best_bot]

# Print results
print("Average Scores for each bot:")
for bot, score in average_scores.items():
    print(f"{bot}: {score:.6f}")

print(f"\nMost relevant bot: {best_bot}")
print(f"Highest average score: {best_score:.6f}")

Average Scores for each bot:
Customer Database Search: 0.813200
Organizational Information: 0.808924
Internet Search: 0.807486

Most relevant bot: Customer Database Search
Highest average score: 0.813200
